# Calibration and uncertainty propagation
Up until now we have been creating models that may accurately represent the local epidemic,
but (at best) only provide one possible epidemic profile 
that would be consistent with our empiric observations of the epidemic. 
In this notebook, we extend this to obtain a range of parameter values and epidemic trajectories 
that would be consistent with the local observations, 
and thereby quantify the uncertainty in our simulations.

Here we will learn how to use a Markov Chain Monte Carlo (MCMC) algorithm 
to calibrate an SIR model to epidemic data.
That is, we will use a Bayesian sampling approach to estimate model parameters 
and to project the epidemic with uncertainty.
Specifically, we will implement the Metropolis algorithm which is one type of MCMC.

The field of Bayesian inference is large and well-established and includes many 
sophisticated algorithms which are outside the scope of this series.
In this notebook, we'll just work through one of the most classical algorithms 
and encourage the reader to explore others 
through the copious resources available for learning 
Bayesian inference more generally.

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

## Calibration data
First, we'll create some dummy data we want our model to fit to.
We'll use data points that look loosely like a bell,
and so could plausibly represent an observed infectious disease epidemic.

In [ ]:
data = pd.DataFrame({"active_cases":
{
    60.0: 3000.0,
    80.0: 8500.0,
    100.0: 21000.0,
    120.0: 40000.0,
    140.0: 44000.0,
    160.0: 30000.0,
    180.0: 16000.0,
    200.0: 7000.0,
}}
)
data["active_cases"].plot(kind="scatter")

## Model

We also need to define our mechanistic model 
of infectious disease transmission,
and we'll just use the same one as we've been using throughout this series.

In [ ]:
def get_sir_model(
    config: dict,
) -> CompartmentalModel:

    model = CompartmentalModel(
        times=(0.0, config["end_time"]),
        compartments=(
            "susceptible", 
            "infectious", 
            "recovered",
        ),
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        },
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery",
        fractional_rate=Parameter("recovery"),
        source="infectious",
        dest="recovered",
    )
    
    return model

## Trial run
Next, let's just quickly run the model and get a sense 
for how the outputs look with some fairly arbitrary starting parameters.
We'll get into adjusting these parameters later in the notebook.
The initial fit is poor,
so it should be pretty obvious if our algorithm is improving on this.

In [ ]:
model_config = {
    "population": 1e6,
    "seed": 100.0,
    "end_time": 365.0,
}
parameters = {
    "contact_rate": 0.3,
    "recovery": 0.1,
}

In [ ]:
sir_model = get_sir_model(model_config)
sir_model.run(parameters)

pd.DataFrame(
    {
        "modelled": sir_model.get_outputs_df()["infectious"],
        "observed": data["active_cases"],
    }
).plot(kind="scatter", labels={"index": "time", "value": "number infectious"})

# Calibration
Now we come to calibration, 
for which we'll need to define a few quantities 
and run through a bit of series to get started.

## Posterior distribution
The main objective of our calibration is to estimate the **_posterior distribution_** of the calibrated parameters. This is the probability distribution of the parameters that are able to describe our observations, given some prior knowledge about these parameters and a mathematical model. In other words, this tells us "What values the parameters should take such that our model is able to capture the data, and given any prior information we had about the parameters before even running the model".

The following graph gives an overview of the posterior computation:
![title](../images/calibration.png)

## Bayes' Theorem

We'll use Bayes' Theorem to decide parameter acceptance.
The posterior probability of a parameter set $\theta$ associated with the data $y$ is denoted $P(\theta | y)$.

Let's write the Bayes' Therorem for reference:
$$P(\theta | y) = \frac{P(y | \theta) \times P(\theta)}{P(y)} \quad.$$

Within the MCMC loop, we are principally interested in the "acceptance ratio" that defines the probability of acceptance of a newly proposed parameter set $\theta '$, when the last accepted parameter set was $\theta$. This is the ratio of the posterior probabilities between $\theta '$ and $\theta$:
$$H := \frac{P(\theta ' | y)}{P(\theta | y)} = \frac{\frac{P(y | \theta ') \times P(\theta ')}{P(y)}}{\frac{P(y | \theta) \times P(\theta)}{P(y)}} = \frac{P(y | \theta ') \times P(\theta ')}{P(y | \theta) \times P(\theta)} \quad .$$

If $H \geq 1$, we'll automatically accept the proposed parameter set $\theta'$. 
Otherwise, the proposed parameter set $\theta'$ will be accepted with probability $H$. 

Here we will define the fundamental aspects of our MCMC calibration:
- Our prior knowledge about the calibrated parameters: $P(\theta)$
- The likelihood associated with our model. This is the probability of observing the data under a given model parameterisation: $P(y|\theta)$

... and some other technical aspects:
- Intitial point from which the MCMC algorithm starts
- The proposal function (or jumping process), defining how we move around in our parameter space. This is defined by $\pi(\theta' | \theta)$ which is the probability of reaching the parameter set $\theta'$, when starting from the parameter set $\theta$.

### Log-transformed quantities
The prior and likelihood quantities are often extremely small numbers in practice, 
which may make computation difficult due to computer precision limits. 
To avoid issues related to rounding, 
the probabilities are usually transformed 
using the logarithm function before calculation of the acceptance ratio.

$$H=\frac{P(y | \theta ') \times P(\theta ')}{P(y | \theta) \times P(\theta)} \\
= exp\Bigl( \Bigl[\ln\bigl(P(y|\theta')\bigr) + \ln\bigl(P(\theta')\bigr) \Bigr] - \Bigl[\ln\bigl(P(y|\theta)\bigr) + \ln\bigl(P(\theta)\bigr) \Bigr] \Bigr) \\
= exp( A(\theta') - A(\theta)) \quad,
$$
where $A(\theta):=\ln\bigl(P(y|\theta)\bigr) + \ln\bigl(P(\theta)\bigr) $ will be referred to as the acceptance quantity associated with parameter set $\theta$.

### Prior distributions

In [ ]:
def evaluate_log_priors(
    proposed_parameters: dict,
) -> float:
    
    # Initialise the prior to 1
    prior_log_proba = 0.0

    # Use a uniform prior on [0., 0.5] for the contact_rate 
    prior_log_proba += stats.uniform.logpdf(x=proposed_parameters["contact_rate"], loc=0.0, scale=0.5)

    # Use a normal prior for the infection duration, with mode ten days
    prior_log_proba += stats.norm.logpdf(x=proposed_parameters["recovery"], loc=0.1, scale=0.1)

    return prior_log_proba

### Likelihood function

In [ ]:
def evaluate_log_likelihood(
    model: CompartmentalModel, 
    proposed_parameters: dict,
) -> float:

    # Build and run the model with the selected parameters
    parameter_set = dict(proposed_parameters)
    model.run(parameter_set)
    modelled_active = model.get_outputs_df()["infectious"]

    # Calculate the log-likelihood associated with the model run
    log_likelihood = 0.0
    for data_time, data_value in data["active_cases"].iteritems():
        modelled_value = modelled_active.loc[data_time]
        
        # Use a normal likelihood with sd=100, centered on the model estimate
        log_likelihood += stats.norm.logpdf(x=data_value, loc=modelled_value, scale=1e4)

    return log_likelihood

### Acceptance quantity
As introduced above, 
this is calculated as the sum of the log prior and the log likelihood.

In [ ]:
def evaluate_acceptance_quantity(
    model: CompartmentalModel, 
    proposed_parameters: dict,
) -> float:

    log_prior = evaluate_log_priors(proposed_parameters)
    log_likelihood = evaluate_log_likelihood(model, proposed_parameters)
    return log_prior + log_likelihood

### Proposal (jumping) function
Get the next parameter set ($\theta'$) from the previous one ($\theta$).

In [ ]:
def propose_parameter_set(
    previous_parameters: dict, 
    jumping_sds: dict,
) -> dict:
    
    proposed_parameters = {}
    for param_name in ["contact_rate", "recovery"]:
        proposed_parameters[param_name] = stats.norm.rvs(
            loc=previous_parameters[param_name], 
            scale=jumping_sds[param_name]
        )

    return proposed_parameters

## The Metropolis algorithm
Now that we have all the pieces, 
we can put them all together in one loop to create the full algorithm.

In [ ]:
def sample_with_metropolis(
    model: CompartmentalModel, 
    n_iter: int, 
    initial_parameters: dict, 
    jumping_sds:dict,
) -> dict:
   
    mcmc_record = pd.DataFrame(
        index=range(n_iter), 
        columns=list(parameters.keys()) + ["acceptance_quantity", "changed_position"],
        dtype=float
    )

    current_parameters = initial_parameters
    current_acceptance_quantity = evaluate_acceptance_quantity(model, current_parameters) 

    for i_run in range(n_iter):
        
        # Print a periodic update so we know things are running...
        if (i_run % 1000) == 0:
            print(f"Running iter {i_run} of {n_iter} iterations")
        
        # Propose a new parameter set and evaluate its acceptance quantity
        proposed_parameters = propose_parameter_set(current_parameters, jumping_sds)
        proposed_acceptance_quantity = evaluate_acceptance_quantity(model, proposed_parameters) 

        # Decide whether to accept the proposed parameters or not
        if proposed_acceptance_quantity >= current_acceptance_quantity:
            accept = 1
        else:
            accept_proba = np.exp(proposed_acceptance_quantity - current_acceptance_quantity)
            accept = stats.binom.rvs(1, accept_proba)  # Flip a coin
        
        # Update the MCMC sampler in case of acceptance
        if accept == 1:
            current_parameters = proposed_parameters
            current_acceptance_quantity = proposed_acceptance_quantity

        # Record the current state
        mcmc_record.loc[i_run] = {
            "contact_rate": current_parameters["contact_rate"], 
            "recovery": current_parameters["recovery"], 
            "acceptance_quantity": float(current_acceptance_quantity),
            "changed_position": accept
        }
    
    return mcmc_record 

# Let's calibrate our model

## Run the algorithm

In [ ]:
n_iterations = 5000
initial_parameters = {
    "contact_rate": 0.15,
    "recovery": 0.1,
}
jumping_sds = {
    "contact_rate": 0.01,
    "recovery": 0.01,
}

mcmc_output = sample_with_metropolis(sir_model, n_iterations, initial_parameters, jumping_sds)

## Explore the MCMC outputs

### Acceptance rate

In [ ]:
n_accepted = mcmc_output["changed_position"].sum()
acceptance_perc = 100.0 * n_accepted / n_iterations
print(f"Our MCMC's acceptance rate is: {round(acceptance_perc,2)}%")

### Progression of the acceptance quantity 

In [ ]:
mcmc_output["acceptance_quantity"].plot()

### Parameter traces
With our output dataframe,
we can easily see the progression of the parameter
values over successive MCMC iterations.

In [ ]:
mcmc_output["contact_rate"].plot()

In [ ]:
mcmc_output["recovery"].plot()

### Burn-in
We want to discard the parameter sets sampled before convergence.

In [ ]:
burn_in = round(n_iterations / 2.0)
post_burn_in_mcmc_output = mcmc_output[burn_in:]

### Posterior distributions
With our output dataframes,
we can easily inspect the distribution of the parameter samples.

In [ ]:
post_burn_in_mcmc_output["contact_rate"].plot.hist()

In [ ]:
post_burn_in_mcmc_output["recovery"].plot.hist()

In [ ]:
post_burn_in_mcmc_output.plot.scatter(x="contact_rate", y="recovery", color="acceptance_quantity")

### The best parameter set
That is, the parameter set with the highest posterior value.

In [ ]:
best_run_id = post_burn_in_mcmc_output["acceptance_quantity"].idxmax()
best_parameters = {
    "contact_rate": post_burn_in_mcmc_output.loc[best_run_id]["contact_rate"],
    "recovery": post_burn_in_mcmc_output.loc[best_run_id]["recovery"],
}
sir_model.run(best_parameters)
pd.DataFrame(
    {
        "modelled": sir_model.get_outputs_df()["infectious"],
        "observed": data["active_cases"],
    }
).plot(kind="scatter")

### Plot sample of model runs
Select 100 accepted parameter sets at random,
run the model with these values 
and plot these against the data.

In [ ]:
n_samples = min(100, n_iterations - burn_in)
sampled_df = post_burn_in_mcmc_output.sample(n_samples)

sampled_output_df = pd.DataFrame(index=sir_model.get_outputs_df().index)
for i_run in sampled_df.index:
    selected_parameters = {
        "contact_rate": post_burn_in_mcmc_output.loc[i_run]["contact_rate"],
        "recovery": post_burn_in_mcmc_output.loc[i_run]["recovery"],
    }
    sir_model.run(selected_parameters)
    sampled_output_df[i_run] = sir_model.get_outputs_df()["infectious"]

In [ ]:
pd.options.plotting.backend = "matplotlib"

fig = sampled_output_df.plot(legend=None, figsize=(15, 8))
fig.plot(data.index, data["active_cases"], marker=".", lw=0, ms=10, color="k")

## Some further considerations
We have presented a very simple implementation of an MCMC-based calibration.
However, in practice, there are quite a few other practical considerations
when undertaking a Bayesian analysis. These include:

- **Use of other MCMC algorithms.** Here we have implemented a "simple" Metropolis-Hastings algorithm, which is the simplest version of MCMCs.
However, there exist other types of MCMCs including Gibbs sampling and the Hamiltonian Monte-Carlo. 
There are also other Bayesian sampling methods that don't verify the Markov property (i.e. not MCMC) 
but can be used for the same purpose. 
This includes adaptive Metropolis samplers such as the Haario algorithm.

- **Use of multiple MCMC chains.** We have only implemented a single MCMC chain that explores the parameter set and samples posterior estimates. 
In practice, it is common to use multiple chains 
that can be run in parallel to generate more samples in the same period of time. 
Samples from the different chains are then combined 
and we can perform statistical tests to check for convergence 
and consistency between the chains (e.g. R-hat statistic).

- **Non-symmetric proposal function.** We have used a symmetric proposal (jumping) function in this example. This means that $\pi(\theta'|\theta) = \pi(\theta|\theta')$.
If the proposal function is not symmetric, we should adjust the acceptance ratio as follows:
$$ H= \frac{P(y | \theta ') \times P(\theta ') \times \pi(\theta|\theta') }{P(y | \theta) \times P(\theta) \times \pi(\theta' |\theta)} \quad .$$

- **Parameter transformation.** When parameter supports are bounded (e.g. finite interval), 
we often transform the parameters into quantities that are unbounded to make sampling easier. 
For example, with the transformed parameter space, 
we don't have to worry about having a proposal function defined on a bounded support. 
These transformations imply some more adjustments to the acceptance ratio that are not discussed here.

- **Thinning.** The samples generated by some MCMC algorithms (e.g. Metropolis-Hastings) are often highly auto-correlated. This is due to the iterative way in which the samples are generated. To address this issue we often apply thinning after generating the samples. That is, we only retain every n-th sampled paramerer sets.

- **Algorithm tuning.** This is probably the most challenging aspect of the Metropolis-Hastings sampler. This is about finding an adequate proposal (jumping) function that will ensure an exhaustive and efficient exploration of the parameter space. If transitions (or jumps) are too big, we will rarely accept the proposed parameters because they would be outside the high-density regions. If transitions are too small, we may not explore the parameter space comprehensively because we may always stay in the same regions. There is no pre-defined rule about how to define a "good" proposal function, but we often want to achieve an acceptance rate of about 10-40%.

## How we actually approach calibration
Having shown this manual implementation of the Metropolis algorithm,
we should stresss that this is actually not how we typically calibrate models.
There are multiple libraries that handle Bayesian sampling with MCMC algorithms already implemented.
These libraries have a range of additional features, 
such as self-tuning and automatic parameter transformation, 
that can address several of the issues listed above.